In [1]:
import pandas as pd
import random
import numpy as np
import time
import copy

### Data clensing

In [2]:
# load file
df = pd.read_excel('test.xlsm')
#df=pd.DataFrame(df)
#df[df['Unnamed: 2']==4]  # select Unnamed: 2 value ==4 
# rename columns and index
df = df.rename(columns={df.columns[0]:'Name',
                        df.columns[1]:'Code',
                        df.columns[2]:'Type',
                        df.columns[3]:'Holiday',
                        df.columns[4]:'Weekday'})
df = df.rename(index={0:'Weekday_ch',1:'Weekday_num',2:'is_holiday'})
# unselect unnamned coluns
unselect_unnamed = [col for col in df if 'Unnamed' not in str(col)]  # 注意 有些還是 date type, 所以用 str
df = df[unselect_unnamed]  

# 有哪些班要運算，determine type_to_generate
type_to_generate = [int(i) for i in str(df['Name'].loc['is_holiday']) if i in '134567890']
df['Name'].loc['is_holiday'] = np.nan  # set value as NaN, not to interfere with count of hollidays


# 將住院醫師簡碼 (iloc[4] and below)以下 code 轉為 str
# 使用 loc 賦值不會出現  SettingWithCopyWarning: 
# A value is trying to be set on a copy of a slice from a DataFrame 
for i in range(4,len(df.index)):
    df.loc[i]['Code'] = str(df.loc[i]['Code'])


# rename date index from 1 to date
for i in range(5,len(df.columns)):
    df = df.rename(columns={df.columns[i]:str(i-4)}) 
    

## ==== now the data has been clensed === ##

df

,Name,Code,Type,Holiday,Weekday,1,2,3,4,5,...,21,22,23,24,25,26,27,28,29,30
Weekday_ch,6,月,NaN,NaN,NaN,一,二,三,四,五,...,日,一,二,三,四,五,六,日,一,二
Weekday_num,NaN,NaN,NaN,NaN,NaN,1,2,3,4,5,...,7,1,2,3,4,5,6,7,1,2
is_holiday,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,v,NaN,NaN,NaN,NaN,NaN,v,v,NaN,NaN
3,NaN,簡,類,假,平,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,李志謙,A,4,2,7,NaN,NaN,NaN,NaN,x,...,NaN,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN
5,戴維安,B,4,2,7,NaN,NaN,NaN,NaN,x,...,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,吳嘉紘,C,4,1,9,NaN,NaN,NaN,NaN,NaN,...,NaN,x,NaN,NaN,NaN,x,NaN,NaN,NaN,x
7,郭昱,D,4,3,9,NaN,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
8,李志謙3,31,3,2,7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN
9,戴維安3,32,3,2,7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,x,NaN


In [3]:
def is_violation(df, duty_type_array):
    """
    input = dataframe after clensing
    output = if there's no day violation
    print where is the violation
    True -> have violation
    False -> no violation
    """
    # 檢查欲執行項目是否為空白
    if duty_type_array == []:
        raise AssertionError('請輸入要執行的班別')
    
        
    TYPES_OF_DUTY = {1:'CT/MR', 3:'ER', 4:'CR', 5:'VS', 6:'Other', 7:'Other', 8:'Other', 9:'Other', 0:'Test'}
    
    is_violation = False
    
    # 檢查每一個要 run 的值班類別班數和是否正確
    for duty_type in duty_type_array:
        # test for holiday
        # Weekday_num 非零的欄位代表當月日數
        days_in_month = df.loc['Weekday_num'].notnull().sum()
        num_of_holiday = df.loc['is_holiday'].notnull().sum()
        num_of_weekday = days_in_month - num_of_holiday

        # if CT/MR, type1 -> doubles the days of dutese
        num_of_holiday = num_of_holiday *2 if duty_type ==1 else num_of_holiday
        num_of_weekday = num_of_weekday *2 if duty_type ==1 else num_of_weekday

        # 假日及平日值班數
        num_of_duties_h = df[df['Type']==duty_type]['Holiday'].sum()
        num_of_duties_w = df[df['Type']==duty_type]['Weekday'].sum()


        # 檢查值班總數是否不足， report error message and violation
        if num_of_duties_h < num_of_holiday:
            print(f'{TYPES_OF_DUTY[duty_type]} 班假日值班總數不足，缺少{num_of_holiday-num_of_duties_h}班')
            is_violation = True
        if num_of_duties_w < num_of_weekday:
            print(f'{TYPES_OF_DUTY[duty_type]} 班平日值班總數不足，缺少{num_of_weekday-num_of_duties_w}班')
            is_violation = True
            
        # 預約值班前後兩天無法值班，避免 qd，並 update 新表，以利接下來亂數產生
        # 4 to len(df.index) 會指到最後一項列
        # 5 to len(df.columns)-1 會指到最後一欄
        # 處理第一欄
        for i in range(4,len(df.index)):
            if df.iloc[i,5] == 1:
                df.iloc[i,6]='x'
        # 處理中間欄
        for i in range(4,len(df.index)):
            for j in range(6,len(df.columns)-1):
                if df.iloc[i,j]==1:
                    df.iloc[i,(j+1)]='x'
                    df.iloc[i,(j-1)]='x'
        # 處理最後一欄
        for i in range(4,len(df.index)):
            if df.iloc[i,len(df.columns)-1] == 1:
                df.iloc[i,len(df.columns)-2]='x'
        
            
        # 檢查是否有某日所有人都無法值班
        # iterate from '1' to 'end'
        for i in range(1,days_in_month+1):
            # any repeated reservation 
            # 符合的 duty type 中 5至end處的值，有多少x or X
            # 使用 map 
            lower_text = lambda item: str(item).lower()
            num_of_exclude = (df[df['Type']==duty_type].iloc[:,5:][str(i)].map(lower_text)=='x').sum()

            # 該班 R 人數
            num_of_r = len(df[df['Type']==duty_type].index)
            if num_of_exclude >= num_of_r:
                print(f'{TYPES_OF_DUTY[duty_type]} 班{i}號所有人均無法值班')
                is_violation = True       
        
        # 檢查是否有某日有兩個以上的人預約要值班
        # iterate from '1' to 'end'
        for i in range(1,days_in_month+1):
            # any repeated reservation 
            # 符合的 duty type 中 5至end處的值，是1的有多少個
            num_of_reservation = (df[df['Type']==duty_type].iloc[:,5:][str(i)]==1).sum()
            if num_of_reservation>1:
                print(f'{TYPES_OF_DUTY[duty_type]} 班{i}號有超過1人預約要值班')
                is_violation = True
        
    return is_violation, df

    

        

In [4]:

violation, df_updated = is_violation(df,type_to_generate)

CT/MR 班假日值班總數不足，缺少9班
CT/MR 班平日值班總數不足，缺少12班
CT/MR 班23號有超過1人預約要值班
Test 班假日值班總數不足，缺少8班
Test 班平日值班總數不足，缺少22班
Test 班1號所有人均無法值班
Test 班2號所有人均無法值班
Test 班3號所有人均無法值班
Test 班4號所有人均無法值班
Test 班5號所有人均無法值班
Test 班6號所有人均無法值班
Test 班7號所有人均無法值班
Test 班8號所有人均無法值班
Test 班9號所有人均無法值班
Test 班10號所有人均無法值班
Test 班11號所有人均無法值班
Test 班12號所有人均無法值班
Test 班13號所有人均無法值班
Test 班14號所有人均無法值班
Test 班15號所有人均無法值班
Test 班16號所有人均無法值班
Test 班17號所有人均無法值班
Test 班18號所有人均無法值班
Test 班19號所有人均無法值班
Test 班20號所有人均無法值班
Test 班21號所有人均無法值班
Test 班22號所有人均無法值班
Test 班23號所有人均無法值班
Test 班24號所有人均無法值班
Test 班25號所有人均無法值班
Test 班26號所有人均無法值班
Test 班27號所有人均無法值班
Test 班28號所有人均無法值班
Test 班29號所有人均無法值班
Test 班30號所有人均無法值班


### Construct the list by condition

In [7]:
preliminary_list = preliminary_gen(df_updated,4)

正在建立 type[4,CR] 初步清單...
type [4,CR] 初步清單已建立完成


### finding the best solution

In [6]:
def preliminary_gen(df_updated, duty_type):
    """
    input: df_updated or df
    generate: preliminary_list

    """
    TYPES_OF_DUTY = {1:'CT/MR', 3:'ER', 4:'CR', 5:'VS', 6:'Other', 7:'Other', 8:'Other', 9:'Other', 0:'Test'}

    print(f'正在建立 type[{duty_type},{TYPES_OF_DUTY[duty_type]}] 初步清單...')

    IS_HOLIDAY = df_updated.iloc[2,5:].tolist()  # list of holiday 'v' [nan, 'v'...]
    DAYS = len(IS_HOLIDAY) # 這個月有幾天, eg 30
    DAY_LIST = [str(i+1) for i in range(DAYS)] # 這個月的號碼 eg ['1'...'28']
    df_work = df_updated[df['Type']==duty_type]

    # combinding 2 dictionaries: z = {**x, **y}, {1:'1', '1':1}
    DAY_TABLE = {**{(i+1):str(i+1) for i in range(DAYS)}, **{str(i+1):(i+1) for i in range(DAYS)}}

    CODE_LIST = df_work['Code'].tolist() # code list, ['31','32']
    num_holiday = {}
    num_weekday = {}
    available_code ={}

    # how many holiday/weekday for each resident
    for code in CODE_LIST:
        num_holiday[code] = df_work[df_work['Code']==code]['Holiday'].item()
        num_weekday[code] = df_work[df_work['Code']==code]['Weekday'].item()

    # construct available days:
    # excluding 預約不值班
    for day in DAY_LIST:
        available_code[day]= CODE_LIST[:]  # 一定要使用完整拷貝，不然會變成參照，後面會全部都錯誤
        for code in CODE_LIST:
            if df_work[df_work['Code']==code][day].item()=='x':  # 如果預約不值班 == 'x'，則從 available 中移除
                available_code[day].remove(code)
    # including 預約值班
    for day in DAY_LIST:
        for code in CODE_LIST:
            if df_work[df_work['Code']==code][day].item()==1: # 如果預約值班，則移除其他
                available_code[day]=[code]

    # 產生 30000個 符合所有排班規則的 candidate
    preliminary_list = []

    count = 0
    while count <30000:  # generate till count = 30000 candidates
        candidate_list = []
        available_code_gen = copy.deepcopy(available_code)  # not alter original list
        for day in DAY_LIST:
            if available_code_gen[day] == []: # no item can be choosed
                # print('break at', day)
                break
            else:
                add_item = random.sample(available_code_gen[day],1)[0]
                candidate_list.append(add_item)
                if (DAY_TABLE[day]+1)> DAYS:
                    break
                try:
                    available_code_gen[DAY_TABLE[DAY_TABLE[day]+1]].remove(add_item)
                except:  # error if at the last day or no the item is no in the next day
                    pass
        if len(candidate_list)==DAYS:  # 如果得出來的，其實不需要這句，因為都篩選到了，但速度幾乎無差別
            preliminary_list.append(candidate_list)
            count+=1 
    
    print(f'type [{duty_type},{TYPES_OF_DUTY[duty_type]}] 初步清單已建立完成')
    return preliminary_list
    # create a list of all candidates, return preliminary_list


### optimizing the list

In [10]:
optimized_list = optimization_singular(preliminary_list, df_updated, 4)

正在尋找 type[4,CR] 最佳排班...
已完成 type[4,CR] 最佳排班排序


In [11]:
for item in optimized_list:
    print(item)

[['A', 'C', 'B', 'D', 'C', 'A', 'C', 'B', 'A', 'C', 'B', 'D', 'C', 'A', 'B', 'D', 'A', 'B', 'D', 'C', 'B', 'D', 'A', 'B', 'C', 'D', 'A', 'B', 'C', 'D'], 14865, 0.4977712860815109]


In [8]:
def optimization_singular(preliminary_list, df_updated, duty_type):
    """
    input: preliminary_list
    output: list_location_std_sorted, after optimization
    """
    # Optimizing the list
    # 1. minimize the total days of QOD in everyone (如果只選標準差多少人不夠)
    # find min() of days  -> 這幾乎是最好的了，因為幾乎<3，所以 2 不需要

    # 2. minimize standard deviation of days of QOD among others
    #np.array([2,2,2,1]).std(ddof=0)
    # 計算個人值班分散程度（標準差） 的標準差，依照順序排列 （大家分散程度要差不多）
    # 在這個情況下，不可能大家同時標準差都很高，導致標準差的標準差值很小


    TYPES_OF_DUTY = {1:'CT/MR', 3:'ER', 4:'CR', 5:'VS', 6:'Other', 7:'Other', 8:'Other', 9:'Other', 0:'Test'}
    DAYS = len(preliminary_list[0])  # 這個月有幾天
    df_work = df_updated[df['Type']==duty_type]
    CODE_LIST = df_work['Code'].tolist() # code list, ['31','32']
    
    
    print(f'正在尋找 type[{duty_type},{TYPES_OF_DUTY[duty_type]}] 最佳排班...')
    
    

    qod = []  # 各組 qod 的情況
    for n in range(len(preliminary_list)):
        # search for qod (value of location i == value of location i+2)
        list_temp = [preliminary_list[n][i] for i in range(DAYS-2) if preliminary_list[n][i]==preliminary_list[n][i+2]] 
        qod.append(list_temp)
    qod_pd = pd.DataFrame(qod)

    # 1. 找到 QOD 人次最少的組合
    # num of qods in each candidate
    num_of_qod = []
    for i in range(len(preliminary_list)):
        num_of_qod.append(qod_pd.iloc[i].notnull().sum())  # 非0個數 = qod 個數
    min_qod = min(num_of_qod)
    # create index of candidates with minimal qod days in total
    #eg [7376, 11732, 15383, 18130, 20990, 28528, 28785]
    min_qod_index = [index for index,value in enumerate(num_of_qod) if value==min_qod]


    # 2. 每個人分布的標準差 之間的標準差 最小化，取三個
    # 最小 qod 的 candidiate
    min_qod_list = []  
    for index in min_qod_index:
        min_qod_list.append(preliminary_list[index])

    # 由 CODE_LIST 內容依序提出資訊
    list_location_std = []
    for i in range(len(min_qod_list)):
        std_temp = []
        for code in CODE_LIST:
            # location of each code
            temp_list = [location for location,item in enumerate(min_qod_list[i]) if item == code]
            # print(code, temp_list, np.std(temp_list))
            std_temp.append(np.std(temp_list,ddof=0))
        list_location_std.append([min_qod_list[i],
                         min_qod_index[i],
                         np.std(std_temp,ddof=0)])  # the list, location, std value

    # 根據 std value (list_location_std[2]) 來排序
    # sorted_a = sorted(a, key=lambda x: x[1])
    list_location_std_sorted = sorted(list_location_std, key=lambda x:x[2]) # list_location_std[2] is the std value

    # 最多取三個
    if len(list_location_std_sorted)>3:
        list_location_std_sorted=list_location_std_sorted[0:3]

    print(f'已完成 type[{duty_type},{TYPES_OF_DUTY[duty_type]}] 最佳排班排序')
    
    return list_location_std_sorted
